### Simulate chromosome with exon duplications to compare tool outcomes

In [1]:
%load_ext autoreload
%autoreload 2
from fun import *

In [2]:
# read db using the gffutils library
db_filename = '/home/msarrias/dbs/homo_1.db'
db = gffutils.FeatureDB(db_filename, keep_order=True)
annot_dict = get_annotations_dict(db)

In [3]:
# parent_dic, parent_child_dic = create_parent_child_dic(annot_dict)
# dump_parent_child_dict(parent_dic, parent_child_dic,'../files/parent_child_dic_human_01.pkl')

# genes parent - child annotations
parent_dic_gene, parent_child_dic_gene_mRNA = create_parent_child_dic(annot_dict,parent_type = ['gene'],
                                                       child_type = ['mRNA'])

# mRNA parent - child annotations
with open('../files/parent_child_dic_human_01.pkl', 'rb') as handle:
    merged_dict = pickle.load(handle)

In [4]:
# #get genes intron regions 

# mRNA_parent_dic, parents_exon_coverage = get_exon_overlaps(merged_dict['parent_child_dic'], 
#                                                            merged_dict['parent_dic'])

# merged_dict = {'parent_dic':mRNA_parent_dic, 'parents_exon_coverage':parents_exon_coverage}
# with open('../files/mRNA_parents_exon_coverage.pkl', 'wb') as handle:
#     pickle.dump(merged_dict, handle)
    
with open('../files/mRNA_parents_exon_coverage.pkl', 'rb') as handle:
    exon_merged_dict = pickle.load(handle)

In [5]:
# get gene - mRNA-exons relationship
gene_hierarchy_dict = {}
for ID, value in parent_child_dic_gene_mRNA.items():
    dict_temp = {}
    for transcript, attrib in value['mRNA'].items():
        dict_temp[transcript] = exon_merged_dict['parents_exon_coverage'][transcript].copy()
    gene_hierarchy_dict[ID] = dict_temp

In [6]:
# parse chromosome seq
fasta_GRCh38_seq = SeqIO.parse(open('/home/msarrias/data/Homo_sapiens.GRCh38.dna.chromosome.1.fa'),'fasta')
fasta_GRCh38_seq = { fasta.id : {'+' : str(fasta.seq),
                                 '-': str(fasta.seq.reverse_complement())} for fasta in fasta_GRCh38_seq}

In [7]:
# get a sample of genes with 3 exons
sample_genes_3exons = {}
gene_hierarchy_dict_cp = copy.deepcopy(gene_hierarchy_dict)

for gene_id, gene_attrib in gene_hierarchy_dict_cp.items():
    # to avoid choosing between transcripts
    if len(gene_attrib) == 1:
        for transcript_key, transcript_dict in gene_attrib.items():
            temp_transcript = exon_merged_dict['parent_dic'][transcript_key]
            if temp_transcript['exon'][0] == 3 and transcript_dict['parent']['strand'] == '+':
                temp_gene_attrib = copy.deepcopy(transcript_dict)
                start, end = transcript_dict['parent']['coord']
                strand = transcript_dict['parent']['strand']
                temp_gene_attrib['parent']['seq'] = get_seq(strand,
                                                            fasta_GRCh38_seq['1'],
                                                            start, end) 
                
                for idx, comp in enumerate(transcript_dict['structure']):
                    for key, value in comp.items():
                        start, end = value['coord']
                        temp_gene_attrib['structure'][idx][key]['seq'] = get_seq(strand, 
                                                                                 fasta_GRCh38_seq['1'],
                                                                                 start, end)
                sample_genes_3exons[gene_id] = temp_gene_attrib

In [8]:
exon_insertion_length = 0
duplicate_exon_n = 2
insert_intron = 'before'
simulated_genes = {}
exon_dup_len_acc = 0
for gene_id, hierarchy_dict in sample_genes_3exons.items():
    exon_count = 0
    sim_gene_coord = []
    sim_gene_comp = []
    for idx, value in enumerate(hierarchy_dict['structure']):
        id_ = list(value.keys())[0]
        value = list(value.values())[0]
        sim_gene_comp.append(id_)
        sim_gene_coord.append(value['coord'])
        if value['type'] == 'exon':
            exon_count += 1
        if exon_count == duplicate_exon_n:
            if insert_intron == 'before':
                comp = copy.deepcopy(hierarchy_dict['structure'][idx-1])
            if insert_intron == 'after':
                comp = copy.deepcopy(hierarchy_dict['structure'][idx+1])
            comp_id = list(comp.keys())[0]
            comp = list(comp.values())[0]
            if comp['type'] == 'intron':
                start, end = comp['coord']
                center = (end - start) // 2
                # we want to leave an evolutionary marker so we insert the 
                # exon duplication leaving 10 bp in the start and end of the intron.
                if center > 10:
                    exon_insertion_length += len(value['seq'])
                    temp_list = copy.deepcopy(sim_gene_coord)
                    temp_coords = [( start, start + center), value['coord'], ( start + center, end)]
                    if start > temp_list[-1][0]:
                        sim_gene_coord.extend(temp_coords)
                    else:
                        for coord_idx, coord in enumerate(temp_list):
                            if start < coord[0]:
                                sim_gene_coord = (sim_gene_coord[:(coord_idx -1)] 
                                                  + temp_coords 
                                                  + sim_gene_coord[coord_idx:])
                                break
                    exon_dup_len_acc += len(value['seq'])
                    if insert_intron == 'before':
                        sim_gene_coord.extend([list(value.values())[0]['coord'] for value in hierarchy_dict['structure'][idx+1:]])
                        sim_gene_comp.extend([comp_id])
                        sim_gene_comp.extend([list(value.keys())[0] for value in hierarchy_dict['structure'][idx:]])
                    if insert_intron == 'after':
                        sim_gene_coord.extend([list(value.values())[0]['coord'] for value in hierarchy_dict['structure'][idx+2:]])
                        sim_gene_comp.extend([comp_id, id_, comp_id])
                        sim_gene_comp.extend([list(value.keys())[0] for value in hierarchy_dict['structure'][idx+2:]])
                else:
                    print('the intron is too short to leave an evolutionary marker')
            else:
                print('two consecutive exons')
            break 
    seq = copy.deepcopy(fasta_GRCh38_seq['1']['+'][(sim_gene_coord[0][0]-1):(sim_gene_coord[0][1]-1)])
    for sim_coord in sim_gene_coord[1:]:
        seq += copy.deepcopy(fasta_GRCh38_seq['1']['+'][(sim_coord[0] - 1):(sim_coord[1]-1)])
    simulated_genes[gene_id] = [hierarchy_dict['parent']['coord'], sim_gene_comp, sim_gene_coord, seq]
    

In [9]:
ch_seq_with_dipl = ''
end = 0
for idx, (sim_gene, sim_gene_list) in enumerate(simulated_genes.items()):
    start, e = sim_gene_list[0]
    temp = (copy.deepcopy(fasta_GRCh38_seq['1']['+'][end:start]) + sim_gene_list[-1])
    if  idx < (len(simulated_genes)-1):
        ch_seq_with_dipl += temp
    else:
        ch_seq_with_dipl += (temp + fasta_GRCh38_seq['1']['+'][e:])
    _ , end = sim_gene_list[0]  

In [10]:
record = SeqRecord(
    Seq(ch_seq_with_dipl),
    id = 'GRCh38_with_exon_dup',
    description="first chromosome in which 22 genes on the "
                "positive strand have duplications of the 2nd exon"
                "inserted in the 1st intron",
)
with open("GRCh38_with_exon_dup.fa", "w") as handle:
    SeqIO.write(record, handle, "fasta")

## Collect the protein sequences of the genes with duplications

In [11]:
## Retrieve a fasta file with the protein sequences of all genes in human.
#!wget http://ftp.ensembl.org/pub/release-107/fasta/homo_sapiens/pep/Homo_sapiens.GRCh38.pep.all.fa.gz
#!gunzip *.gz

In [12]:
human_protein_seqs = {}
fasta_sequences = SeqIO.parse(open("Homo_sapiens.GRCh38.pep.all.fa"),'fasta')
for fasta in fasta_sequences:
    gene = [i for i in fasta.description.rsplit(' ') if 'gene:' in i]
    if gene:
        if gene[0] not in human_protein_seqs:
            human_protein_seqs[gene[0]] = {}
        human_protein_seqs[gene[0]][fasta.id] = str(fasta.seq)

In [14]:
proteins_simulated_genes = {}
with open("genes_with_dup_original_prot_seq.fa", "w") as handle:
    for gene_id in simulated_genes.keys():
        gene_id_prot_dict = [i for i in list(human_protein_seqs.keys()) if gene_id in i]
        if gene_id_prot_dict:
            temp = copy.deepcopy(human_protein_seqs[gene_id_prot_dict[0]])
            for prot_id, seq in temp.items():  
                record = SeqRecord(Seq(seq),
                                   id = str(prot_id),
                                  description= gene_id_prot_dict[0])
                SeqIO.write(record, handle, "fasta")
            proteins_simulated_genes[gene_id_prot_dict[0]] = temp